In [2]:
import yfinance as yf
import pandas as pd
import datetime
from alpaca.data import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Set up Alpaca API credentials
ALPACA_API_KEY = os.getenv('ALPACA_API_KEY')
ALPACA_SECRET_KEY = os.getenv('ALPACA_SECRET_KEY')
# print(ALPACA_API_KEY)

# Initialize Alpaca Data and Trading clients
data_client = StockHistoricalDataClient(ALPACA_API_KEY, ALPACA_SECRET_KEY)
trading_client = TradingClient(ALPACA_API_KEY, ALPACA_SECRET_KEY, paper=True)

# Example to fetch real-time bars for a symbol using Alpaca
def get_real_time_stock_data(symbol):
    start = datetime.datetime.now() - datetime.timedelta(days=1)
    end = datetime.datetime.now()

    request_params = StockBarsRequest(
        symbol_or_symbols=symbol,
        timeframe=TimeFrame.Minute,
        start=start,
        end=end
    )
    bars = data_client.get_stock_bars(request_params).df
    return bars

# Function to fetch stock data using yfinance
def get_stock_data(stock_symbol):
    """Fetch daily stock data using yfinance."""
    try:
        stock = yf.Ticker(stock_symbol)
        hist = stock.history(period='1y')
        dates = hist.index.strftime('%Y-%m-%d').tolist()
        prices = hist['Close'].tolist()
        return dates, prices
    except Exception as e:
        print(f"Error fetching stock data for {stock_symbol}: {e}")
        return None, None

# Function to fetch options data using yfinance
def get_options_data(stock_symbol):
    """Fetch options data using yfinance."""
    try:
        stock = yf.Ticker(stock_symbol)
        expiration_dates = stock.options
        options_data = {}
    
        for date in expiration_dates:
            calls = stock.option_chain(date).calls
            puts = stock.option_chain(date).puts
            options_data[date] = {
                'calls': calls[['strike', 'lastPrice', 'bid', 'ask', 'volume']],
                'puts': puts[['strike', 'lastPrice', 'bid', 'ask', 'volume']]
            }
        return options_data
    except Exception as e:
        print(f"Error fetching options data for {stock_symbol}: {e}")
        return {}

# Fetch account information using Alpaca
def get_account():
    try:
        account = trading_client.get_account()
        return account
    except Exception as e:
        print(f"Error fetching account data: {e}")

# Fetch positions using Alpaca
def get_positions():
    try:
        positions = trading_client.get_all_positions()
        return positions
    except Exception as e:
        print(f"Error fetching positions: {e}")

# Fetch portfolio summary
def get_portfolio():
    account_info = get_account()
    positions = get_positions()

    if account_info and positions:
        portfolio_summary = {
            "cash": float(account_info.cash),
            "equity": float(account_info.equity),
            "buying_power": float(account_info.buying_power),
            "positions": positions
        }
        return portfolio_summary
    return {}

# Place order using Alpaca
def place_order(symbol, qty, side):
    try:
        market_order_data = MarketOrderRequest(
            symbol=symbol,
            qty=qty,
            side=OrderSide.BUY if side == 'buy' else OrderSide.SELL,
            time_in_force=TimeInForce.GTC
        )
        market_order = trading_client.submit_order(market_order_data)
        return market_order
    except Exception as e:
        print(f"Error placing order for {symbol}: {e}")

# Fetch order history using Alpaca
def get_order_history():
    try:
        orders = trading_client.get_orders(status='all', limit=50)
        return orders
    except Exception as e:
        print(f"Error fetching order history: {e}")

# Fetch recent trades/orders using Alpaca
def get_recent_trades():
    try:
        orders = trading_client.get_orders(status='all', limit=50)
        return orders
    except Exception as e:
        print(f"Error fetching recent trades: {e}")

# Function to test getting historical stock data and options data
def test_stock_data(symbol):
    print(f"Testing stock data for {symbol}...")
    dates, prices = get_stock_data(symbol)
    print(f"Dates: {dates[:5]}")
    print(f"Prices: {prices[:5]}")

    print(f"Testing options data for {symbol}...")
    options = get_options_data(symbol)
    print(f"Options data: {list(options.keys())[:5]}")

# Function to test fetching real-time stock data
def test_real_time_data(symbol):
    print(f"Testing real-time data for {symbol}...")
    real_time_data = get_real_time_stock_data(symbol)
    print(real_time_data.head())

# Function to test Alpaca trading API (account, portfolio, positions)
def test_alpaca_functions():
    print("Testing Alpaca API...")
    account = get_account()
    print(f"Account info: {account}")

    positions = get_positions()
    print(f"Positions: {positions}")

    portfolio = get_portfolio()
    print(f"Portfolio: {portfolio}")

# Run tests
if __name__ == "__main__":
    # Test yfinance data fetching
    test_stock_data('AAPL')

    # Test real-time stock data
    test_real_time_data('AAPL')

    # Test Alpaca trading API
    test_alpaca_functions()


Testing stock data for AAPL...
Dates: ['2023-10-19', '2023-10-20', '2023-10-23', '2023-10-24', '2023-10-25']
Prices: [174.56671142578125, 171.9998779296875, 172.11924743652344, 172.5570068359375, 170.2289276123047]
Testing options data for AAPL...
Options data: ['2024-10-25', '2024-11-01', '2024-11-08', '2024-11-15', '2024-11-22']
Testing real-time data for AAPL...
                                      open      high     low     close  \
symbol timestamp                                                         
AAPL   2024-10-18 19:09:00+00:00  235.0685  235.2999  235.06  235.2440   
       2024-10-18 19:10:00+00:00  235.2450  235.3000  235.20  235.2300   
       2024-10-18 19:11:00+00:00  235.2300  235.3450  235.23  235.3156   
       2024-10-18 19:12:00+00:00  235.3150  235.4250  235.30  235.4250   
       2024-10-18 19:13:00+00:00  235.4200  235.4800  235.35  235.3549   

                                    volume  trade_count        vwap  
symbol timestamp                           

In [3]:
import yfinance as yf
import pandas as pd
import datetime
from alpaca.data import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Set up Alpaca API credentials
ALPACA_API_KEY = os.getenv('ALPACA_API_KEY')
ALPACA_SECRET_KEY = os.getenv('ALPACA_SECRET_KEY')

# Initialize Alpaca Data and Trading clients
data_client = StockHistoricalDataClient(ALPACA_API_KEY, ALPACA_SECRET_KEY)
trading_client = TradingClient(ALPACA_API_KEY, ALPACA_SECRET_KEY, paper=True)

# Example to fetch real-time bars for a symbol using Alpaca
def get_real_time_stock_data(symbol):
    start = datetime.datetime.now() - datetime.timedelta(days=1)
    end = datetime.datetime.now()

    request_params = StockBarsRequest(
        symbol_or_symbols=symbol,
        timeframe=TimeFrame.Minute,
        start=start,
        end=end
    )
    bars = data_client.get_stock_bars(request_params).df
    return bars

# Function to fetch stock data using yfinance
def get_stock_data(stock_symbol):
    """Fetch daily stock data using yfinance."""
    try:
        stock = yf.Ticker(stock_symbol)
        hist = stock.history(period='1y')
        dates = hist.index.strftime('%Y-%m-%d').tolist()
        prices = hist['Close'].tolist()
        return dates, prices
    except Exception as e:
        print(f"Error fetching stock data for {stock_symbol}: {e}")
        return None, None

# Function to fetch options data using yfinance
def get_options_data(stock_symbol):
    """Fetch options data using yfinance."""
    try:
        stock = yf.Ticker(stock_symbol)
        expiration_dates = stock.options
        options_data = {}
    
        for date in expiration_dates:
            calls = stock.option_chain(date).calls
            puts = stock.option_chain(date).puts
            options_data[date] = {
                'calls': calls[['strike', 'lastPrice', 'bid', 'ask', 'volume']],
                'puts': puts[['strike', 'lastPrice', 'bid', 'ask', 'volume']]
            }
        return options_data
    except Exception as e:
        print(f"Error fetching options data for {stock_symbol}: {e}")
        return {}

# Fetch account information using Alpaca
def get_account():
    try:
        account = trading_client.get_account()
        return account
    except Exception as e:
        print(f"Error fetching account data: {e}")

# Fetch positions using Alpaca
def get_positions():
    try:
        positions = trading_client.get_all_positions()
        return positions
    except Exception as e:
        print(f"Error fetching positions: {e}")

# Fetch portfolio summary
def get_portfolio():
    account_info = get_account()
    positions = get_positions()

    if account_info and positions:
        portfolio_summary = {
            "cash": float(account_info.cash),
            "equity": float(account_info.equity),
            "buying_power": float(account_info.buying_power),
            "positions": positions
        }
        return portfolio_summary
    return {}

# Place order using Alpaca
def place_order(symbol, qty, side):
    try:
        market_order_data = MarketOrderRequest(
            symbol=symbol,
            qty=qty,
            side=OrderSide.BUY if side == 'buy' else OrderSide.SELL,
            time_in_force=TimeInForce.GTC
        )
        market_order = trading_client.submit_order(market_order_data)
        return market_order
    except Exception as e:
        print(f"Error placing order for {symbol}: {e}")

# Fetch order history using Alpaca
def get_order_history():
    try:
        orders = trading_client.get_orders(status='all', limit=50)
        return orders
    except Exception as e:
        print(f"Error fetching order history: {e}")

# Fetch recent trades/orders using Alpaca
def get_recent_trades():
    try:
        orders = trading_client.get_orders(status='all', limit=50)
        return orders
    except Exception as e:
        print(f"Error fetching recent trades: {e}")

# Function to test getting stock, options, and real-time data
def test_stock_and_options_data(symbol):
    print(f"Testing stock data for {symbol}...")
    
    # Fetching stock data
    dates, prices = get_stock_data(symbol)
    if dates and prices:
        print(f"Stock Dates: {dates[:5]}")
        print(f"Stock Prices: {prices[:5]}")
    else:
        print("No stock data available.")
    
    # Fetching options data
    print(f"Testing options data for {symbol}...")
    options = get_options_data(symbol)
    if options:
        print(f"Available Expiration Dates: {list(options.keys())}")
        for expiration, data in options.items():
            print(f"Expiration Date: {expiration}")
            print(f"Call Options Sample: \n{data['calls'].head()}")
            print(f"Put Options Sample: \n{data['puts'].head()}")
            break  # Limiting to first expiration date for brevity
    else:
        print("No options data available.")
    
    # Fetching real-time stock data
    print(f"Testing real-time data for {symbol}...")
    real_time_data = get_real_time_stock_data(symbol)
    if not real_time_data.empty:
        print(real_time_data.head())
    else:
        print(f"No real-time data available for {symbol}.")

# Function to test Alpaca trading API (account, portfolio, positions)
def test_alpaca_functions():
    print("Testing Alpaca API...")
    account = get_account()
    print(f"Account info: {account}")

    positions = get_positions()
    print(f"Positions: {positions}")

    portfolio = get_portfolio()
    print(f"Portfolio: {portfolio}")

# Run tests
if __name__ == "__main__":
    # Test stock, options, and real-time data
    test_stock_and_options_data('AAPL')

    # Test Alpaca trading API
    test_alpaca_functions()


Testing stock data for AAPL...
Stock Dates: ['2023-10-19', '2023-10-20', '2023-10-23', '2023-10-24', '2023-10-25']
Stock Prices: [174.5667266845703, 171.99986267089844, 172.11924743652344, 172.5570068359375, 170.22891235351562]
Testing options data for AAPL...
Available Expiration Dates: ['2024-10-25', '2024-11-01', '2024-11-08', '2024-11-15', '2024-11-22', '2024-11-29', '2024-12-20', '2025-01-17', '2025-02-21', '2025-03-21', '2025-04-17', '2025-06-20', '2025-08-15', '2025-09-19', '2025-12-19', '2026-01-16', '2026-06-18', '2026-12-18', '2027-01-15']
Expiration Date: 2024-10-25
Call Options Sample: 
   strike  lastPrice     bid     ask  volume
0   100.0     126.51  134.25  135.85       5
1   105.0     126.68  129.85  130.35       1
2   130.0     101.74  104.85  105.85       2
3   140.0      95.79   94.30   96.00       1
4   145.0      90.47   89.20   90.95       8
Put Options Sample: 
   strike  lastPrice  bid   ask  volume
0   100.0       0.01  0.0  0.05     2.0
1   105.0       0.06  0